In [ ]:
# use Qt instead of Tk backend (on mac)
import matplotlib
matplotlib.use("Qt4Agg")

import panel as pn

pn.extension('mathjax')

In [ ]:
# general settings:
export_as_html = False

In [ ]:
# start with progress bar
import ipywidgets as widgets
from IPython.display import display


def init_pbar_1(Nmax):
    pbar_1 = widgets.IntProgress(
        value=0,
        min=0,
        max=Nmax,
        step=1,
        description='Loading:',
        bar_style='', # 'success', 'info', 'warning', 'danger' or ''
        orientation='horizontal'
    )
    display(pbar_1)
    return pbar_1

# show progress bar (updated during the course of this script via incrementing pbar.value)
pbar_1 = init_pbar_1(4)

In [ ]:
# load aiida
from aiida import load_profile
profile = load_profile()

In [ ]:
# load other modules
import ipywidgets as widgets
from IPython.display import display
import markdown 
from aiida.orm import load_node
from aiida_kkr.tools import plot_kkr

from aiida_kkr.calculations import VoronoiCalculation
from aiida.orm import StructureData

from ase_notebook import AseView, ViewConfig

In [ ]:
import numpy as np
import bokeh.plotting as bkp
from bokeh.plotting import figure, show, output_notebook, output_file
from bokeh.models import ColumnDataSource
from bokeh.models import Span
from bokeh.models import Label
from bokeh.layouts import row


import ipywidgets as widgets
from IPython.display import display
from ipywidgets import HBox, VBox

In [ ]:
if export_as_html:
    output_file('/Users/ruess/Desktop/Sb2Te3_host_bandstruc_dos.html')
else:
    # set output to notbook not file
    bkp.output_notebook(hide_banner=True)

### Electronic structure of Sb$_2$Te$_3$ host crystal

In [ ]:
# Some general settings:

show_structure = True

In [ ]:
# host system: Sb2Te3 6QL slab
# - DOS and bandstructure
# - crystal structure, highlight impurity positions

# load data from database
Sb2Te3_6QL_DOS = load_node('edb8f4ec-38a0-46c9-9334-4448915aebb1') # +/-5eV
Sb2Te3_6QL_bandstruc = load_node('d3a077c4-967e-41a1-9817-a687e8ab475d') # -0.5eV..+0.3eV, corrected K-path

show_empty_atoms= False

# shift reference energy
EF0 = 0.2

gapend = 0.095    #0.105
gapstart = -0.062  #-0.06

In [ ]:
pbar_1.value += 1

In [ ]:
# Functions to plot crystal structure:

In [ ]:
def prepare_plotting_structure(return_struc=False):
    # find and plot structure
    structure0, voro_calc = VoronoiCalculation.find_parent_structure(Sb2Te3_6QL_bandstruc)

    if structure0.has_vacancies:
        cell = structure0.cell
        
        # correct cell ...
        cell[2] = [i*8 for i in cell[2]]

        stmp = StructureData(cell=cell)
        for site in structure0.sites:
            k = structure0.get_kind(site.kind_name)
            pos = np.array(site.position)
            pos[2] = -pos[2]
            if not k.has_vacancies:
                stmp.append_atom(position=pos, symbols=k.symbol)
            elif show_empty_atoms:
                stmp.append_atom(position=pos, symbols='X')
            #else:
            #    print("removing atom", site)
        stmp.set_pbc(structure0.pbc)
        structure = stmp
        
    if return_struc:
        return structure

    # now construct ase object and use ase's viewer
    ase_atoms = structure.get_ase()
    
    return ase_atoms

In [ ]:
def create_structure_plot(ase_atoms, static_plot=False):
    # set up structure viewer from ase_notebook

    config_dict = {
        'atom_show_label': True, #True,
        'rotations': "-90x,-60y,0z",
        #'rotations': "-90x,-60y,180z",
        'show_uc_repeats': True,
        'show_bonds': False,
        'show_unit_cell': False,
#        'canvas_size': [120, 400],
        'canvas_size': [120, 400],
        'zoom': 1.0,
        'show_axes': True,
        'canvas_background_opacity': 0.05,
        'canvas_color_background': 'black',
        'axes_length': 30,
    }
    
    #ase_atoms.rotate(180, 'z', rotate_cell=True)
    ase_atoms.rotate('-z', 'z', rotate_cell=True)
    #ase_atoms.rotate(-90, 'x', rotate_cell=True)
    #ase_atoms.rotate(-60, 'y', rotate_cell=True)

    config = ViewConfig(**config_dict)
    ase_view = AseView(config)
    ase_view_imp = AseView(config)
    ase_view_imp.config.zoom = 0.7

    
    if not static_plot:
        strucview = ase_view.make_render(
                ase_atoms, center_in_uc=False,
                repeat_uc=(3,3,1), use_atom_arrays=True,
                create_gui=True, #True, 
            )
    else:
        ase_view.config.zoom = 1.2
        ase_view.config.atom_show_label = False
        ase_view.config.show_axes = False
        ase_view.config.canvas_background_opacity = 0.0
        strucview = ase_view.make_svg(ase_atoms, center_in_uc=False,
                repeat_uc=(3,3,1)
             )
        
    return strucview

In [ ]:
# Functions to plot bandstructure

In [ ]:
def load_bandstruc_data():
    if 'saved_data_dispersion.npy' in Sb2Te3_6QL_bandstruc.outputs.retrieved.list_object_names():
        # load data
        import numpy as np
        with Sb2Te3_6QL_bandstruc.outputs.retrieved.open('saved_data_dispersion.npy') as f:
            fn = f.name
        data_bandstruc = np.load(fn)
    else:
        raise ValueError('Could not load bandstructure data')

    with Sb2Te3_6QL_bandstruc.outputs.retrieved.open('output.0.txt') as f:
        ef = [float(i.split('=')[2].split()[0]) for i in f.readlines() if 'Fermi' in i][0]

    ne = len(set(data_bandstruc[:,0]))
    nk = int(len(data_bandstruc)/ne)
    data_bandstruc = data_bandstruc.reshape(ne,nk,-1)


    y = (data_bandstruc[:,0,0] - ef)*13.6 + EF0
    x = data_bandstruc[0,:,2] - data_bandstruc[0,:,3]
    dtot = data_bandstruc[:,:,5]
    
    return x, y, dtot

In [ ]:
def plot_bandstruc():
    x, y, dtot = load_bandstruc_data()

    x_range=[-0.6,0.4] #x.min(),x.max()]
    y_range=[y.min(),y.max()]

    dw = x_range[1]-x_range[0]
    dh = y_range[1]-y_range[0]

    # for paper plot:
    #p = bkp.figure(plot_width=500, plot_height=350,
    p = bkp.figure(plot_width=350, plot_height=400, 
                   tools='pan,box_zoom,wheel_zoom,reset,save',
               #tooltips=[("x", "$x"), ("y", "$y")],
               x_range=x_range, y_range=y_range)

    # must give a vector of image data for image parameter
    p.image(image=[-dtot], x=x_range[0], y=y_range[0], dw=dw, dh=dh, palette="Greys256")


    p.title.text = "Sb2Te3 6QL bandstructure"
    p.yaxis.axis_label = 'E-EF (eV)'

    p.xaxis.ticker = [-0.6, 0, 0.4]
    p.xaxis.major_label_overrides = {-0.6: 'M', 0: 'Gamma', 0.4: 'K'}

    #bkp.show(p)
    return p

In [ ]:
def add_ef_lines_bandstruc(bandstruc_plot):
    # add hbar (blue) to left plot
    bandstruc_plot.hbar(y=gapstart+gapwidth/2, height=gapwidth, left=-0.1*ymax, right=ymax*1.2, color='lightblue', 
           alpha=0.4, name='grey')

    # add lines for other EF positions
    bandstruc_plot.line([-2, 2], [0.2, 0.2], line_width=1, line_alpha=0.8, color='green', legend_label='+200meV')
    bandstruc_plot.line([-2, 2], [-0.2,-0.2], line_width=1, line_alpha=0.8, color='red', legend_label='-200meV')

    # make legend and allow lines to disappear on click
    bandstruc_plot.legend.location = "top_left"
    bandstruc_plot.legend.click_policy="hide"

    # add text to label bulk band gap region
    band_gap_text = Label(x=-0.55, y=-0.01, x_units='data', y_units='data',
                          text='bulk band gap', render_mode='css',
                         )
    bandstruc_plot.add_layout(band_gap_text)

In [ ]:
# Functions to plot host DOS

In [ ]:
def get_data_dos():
    
    # define global variables (needed by add_ef_lines_bandstruc function)
    global ymax, gapwidth

    # load DOS data
    dos_data = Sb2Te3_6QL_DOS.outputs.dos_data_interpol

    xlbl, x, xunit = dos_data.get_x()
    xlbl += ' ('+xunit+')'
    # shift Fermi level
    x += EF0

    ylbl, y, yunit = dos_data.get_y()[0] # total dos
    ylbl += ' ('+yunit+')'

    # find values for gap region
    gapwidth = gapend - gapstart
    ymax = max(np.sum(y[1::2,:], axis=0) - np.sum(y[0::2,:], axis=0))
    
    # create dos source for plotting with bokeh
    #                                     E-EF    (        spin up     )   (     -spin down     )
    source = ColumnDataSource(data=dict(x=x[0], y=np.sum(y[1::2,:], axis=0) - np.sum(y[0::2,:], axis=0),
                                       )
                             )
    
    return source

In [ ]:
def plot_dos():
    
    # prepare DOS data
    source_dos = get_data_dos()

    # open bokeh figure
    # for paper plot:
    #dos_plot = bkp.figure(plot_width=500, plot_height=300, y_range = [0, ymax*1.05], x_range=[-4, 3])
    dos_plot = bkp.figure(plot_width=400, plot_height=400, y_range = [0, ymax*1.05], x_range=[-4, 3],
                          tools='pan,box_zoom,wheel_zoom,reset,save',)

    # set title and axis labels for DOS plot
    dos_plot.title.text = "Sb2Te3 6QL DOS"
    dos_plot.xaxis.axis_label = 'E (eV)'
    dos_plot.yaxis.axis_label = 'DOS (states/eV)'
    
    
    # add vbar (blue) to right plot
    dos_plot.vbar(x=gapstart+gapwidth/2, width=gapwidth, bottom=-0.1*ymax, top=ymax*1.2, color='lightblue', 
            alpha=0.4, name='grey')

    # plot DOS data
    dos_plot.line('x', 'y', source=source_dos, line_width=3, line_alpha=0.6, color='black', legend_label='total DOS')
    
    # add lines
    dos_plot.line([0.2, 0.2], [-200, 200], line_width=1, line_alpha=0.8, color='green', legend_label='+200meV')
    dos_plot.line([-0.2, -0.2], [-200, 200], line_width=1, line_alpha=0.8, color='red', legend_label='-200meV')

    dos_plot.legend.location = "bottom_left"
    dos_plot.legend.click_policy="hide"
    
    dos_plot.grid.visible = False


    #bkp.show(dos_plot)
    return dos_plot

In [ ]:
# Now do plotting ...

In [ ]:
if show_structure:
    # 
    ase_atoms = prepare_plotting_structure()

    # 
    strucview = create_structure_plot(ase_atoms, static_plot=True)


    # show structure viewer
    #strucview

    ipywidgetout_strucview = widgets.Output()

    with ipywidgetout_strucview:
        display(strucview)

    # show strucview viewer in ipywidget output
    #ipywidgetout_strucview

In [ ]:
pbar_1.value += 1

In [ ]:
# DOS and bandstructure plots

In [ ]:
# create bandstructure plot
bandstruc_plot = plot_bandstruc()

In [ ]:
pbar_1.value += 1

In [ ]:
# make DOS plot
dos_plot = plot_dos()
# add ef lines to bandstructure plot (needs to be done here since some values are otherwise not defined)
add_ef_lines_bandstruc(bandstruc_plot)

In [ ]:
#show(dos_plot)

In [ ]:
#show(bandstruc_plot)

In [ ]:
pbar_1.value += 1

In [ ]:
legend = pn.pane.Markdown("## Host system: thin film of Sb$_2$Te$_3$"
                          "\n"
                          "### Legend\n"
                          "* Red line: Fermi level shift into conduction band\n"
                          "* Green line: Fermi level shift into valence band\n"
                          "* blue region: region of bulk band gap\n"
                         )
legend

In [ ]:
# put bandstructure and DOS plots together
layout = row(bandstruc_plot,dos_plot)


#bkp.show(layout)

o2 = widgets.Output()

with o2:
    bkp.show(layout)

if show_structure:
    display_all = HBox([ipywidgetout_strucview, o2])
else:
    display_all = VBox([o2])

if export_as_html:
    # open/ save as html
    bkp.show(layout)
else:
    # open interactively
    #display(display_all)
    #pn.Column(display_all).servable()
    struc_title = pn.pane.LaTeX("Thin film (6QL) of Sb$_2$Te$_3$")
    #host_plots = pn.Row(pn.Column(struc_title, strucview), layout)
    host_plots = pn.Row(strucview, layout)

In [ ]:
pbar_1.close()

In [ ]:
# now show plots
host_plots